# Machine Learnig Bootcamp
# HomeWork 8

# Dataset
In this homework, we'll build a model for classifying various hair types. For this, we will use the Hair Type dataset that was obtained from Kaggle and slightly rebuilt.<br>

You can download the target dataset for this homework from here:<br>

wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip<br>
unzip data.zip<br>
In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch.<br>

We will use PyTorch for that.<br>

You can use Google Colab or your own computer for that.<br>

In [5]:

!unzip -q data.zip

# Data Preparation<br>
The dataset contains around 1000 images of hairs in the separate folders for training and test sets.<br>


# Reproducibility<br>
Reproducibility in deep learning is a multifaceted challenge that requires attention to both software and hardware details. In some cases, we can't guarantee exactly the same <br>results during the same experiment runs.<br>

Therefore, in this homework we suggest to set the random number seed generators by:<br>

import numpy as np<br>
import torch<br>

SEED = 42<br>
np.random.seed(SEED)<br>
torch.manual_seed(SEED)<br>

if torch.cuda.is_available():<br>
    torch.cuda.manual_seed(SEED)<br>
    torch.cuda.manual_seed_all(SEED)<br>

torch.backends.cudnn.deterministic = True<br>
torch.backends.cudnn.benchmark = False<br>
Also, use PyTorch of version 2.8.0 (that's the one in Colab).<br>

In [18]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchsummary import summary

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [17]:
data_dir = "data"

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = train_transforms


In [19]:
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transforms)
validation_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

len(train_dataset), len(validation_dataset)


(800, 201)

# Model<br>
For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.<br>

You need to develop the model with following structure:<br>

The shape for input should be (3, 200, 200) (channels first format in PyTorch)<br>
Next, create a convolutional layer (nn.Conv2d):<br>
Use 32 filters (output channels)<br>
Kernel size should be (3, 3) (that's the size of the filter)<br>
Use 'relu' as activation<br>
Reduce the size of the feature map with max pooling (nn.MaxPool2d)<br>
Set the pooling size to (2, 2)<br>
Turn the multi-dimensional result into vectors using flatten or view<br>
Next, add a nn.Linear layer with 64 neurons and 'relu' activation<br>
Finally, create the nn.Linear layer with 1 neuron - this will be the output<br>
The output layer should have an activation - use the appropriate activation for the binary classification case<br>
As optimizer use torch.optim.SGD with the following parameters:<br>

torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)<br>

In [22]:
class HairNet(nn.Module):
    def __init__(self):
        super(HairNet, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.flatten_dim = 32 * 99 * 99
        
        self.fc = nn.Sequential(
            nn.Linear(self.flatten_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # binary output
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, self.flatten_dim)
        x = self.fc(x)
        return x

model = HairNet().to(device)


# Question 1 <br>
Which loss function you will use?<br>
<br>
nn.MSELoss()<br>
nn.BCEWithLogitsLoss()<br>
nn.CrossEntropyLoss()<br>
nn.CosineEmbeddingLoss()<br>

In [23]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [11]:
nn.BCEWithLogitsLoss()

BCEWithLogitsLoss()

# Question 2 <br>
What's the total number of parameters of the model? You can use torchsummary or count manually.<br>
<br>
In PyTorch, you can find the total number of parameters using:<br>

Option 1: Using torchsummary (install with: pip install torchsummary)<br>
from torchsummary import summary<br>
summary(model, input_size=(3, 200, 200))<br>
<br>
Option 2: Manual counting<br>
total_params = sum(p.numel() for p in model.parameters())<br>
print(f"Total parameters: {total_params}")<br>
896<br>
11214912<br>
15896912<br>
20073473<br>

In [21]:
summary(model, (3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------


# Generators and Training<br>
For the next two questions, use the following transformation for both train and test sets:<br>

train_transforms = transforms.Compose([<br>
    transforms.Resize((200, 200)),<br>
    transforms.ToTensor(),<br>
    transforms.Normalize(<br>
        mean=[0.485, 0.456, 0.406],<br>
        std=[0.229, 0.224, 0.225]<br>
    ) # ImageNet normalization<br>
])<br>
We don't need to do any additional pre-processing for the images.<br>
Use batch_size=20<br>
Use shuffle=True for both training, but False for test.<br>
Now fit the model.<br>
<br>
You can use this code:<br>
<br>
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}<br>

for epoch in range(num_epochs):<br>
    model.train()<br>
    running_loss = 0.0<br>
    correct_train = 0<br>
    total_train = 0<br>
    for images, labels in train_loader:<br>
        images, labels = images.to(device), labels.to(device)<br>
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)<br>
<br>
        optimizer.zero_grad()<br>
        outputs = model(images)<br>
        loss = criterion(outputs, labels)<br>
        loss.backward()<br>
        optimizer.step()<br>

        running_loss += loss.item() * images.size(0)<br>
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy<br>
        predicted = (torch.sigmoid(outputs) > 0.5).float()<br>
        total_train += labels.size(0)<br>
        correct_train += (predicted == labels).sum().item()<br>
<br>
    epoch_loss = running_loss / len(train_dataset)<br>
    epoch_acc = correct_train / total_train<br>
    history['loss'].append(epoch_loss)<br>
    history['acc'].append(epoch_acc)<br>

    model.eval()<br>
    val_running_loss = 0.0<br>
    correct_val = 0<br>
    total_val = 0<br>
    with torch.no_grad():<br>
        for images, labels in validation_loader:<br>
            images, labels = images.to(device), labels.to(device)<br>
            labels = labels.float().unsqueeze(1)<br>
            outputs = model(images)<br>
            loss = criterion(outputs, labels)<br>
<br>
            val_running_loss += loss.item() * images.size(0)<br>
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)<br>
            correct_val += (predicted == labels).sum().item()<br>

    val_epoch_loss = val_running_loss / len(validation_dataset)<br>
    val_epoch_acc = correct_val / total_val<br>
    history['val_loss'].append(val_epoch_loss)<br>
    history['val_acc'].append(val_epoch_acc)<br>

    print(f"Epoch {epoch+1}/{num_epochs}, "<br>
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "<br>
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}"))<br>

In [24]:
num_epochs = 10

history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train

    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    # ------------------ Validation ------------------
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val

    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")


Epoch 1/10, Loss: 0.6466, Acc: 0.6400, Val Loss: 0.6080, Val Acc: 0.6318
Epoch 2/10, Loss: 0.5434, Acc: 0.7388, Val Loss: 0.5840, Val Acc: 0.6468
Epoch 3/10, Loss: 0.4605, Acc: 0.7937, Val Loss: 0.5788, Val Acc: 0.6617
Epoch 4/10, Loss: 0.4275, Acc: 0.8100, Val Loss: 0.5905, Val Acc: 0.6766
Epoch 5/10, Loss: 0.3143, Acc: 0.8650, Val Loss: 0.5940, Val Acc: 0.7512
Epoch 6/10, Loss: 0.2394, Acc: 0.9075, Val Loss: 0.5851, Val Acc: 0.7463
Epoch 7/10, Loss: 0.2045, Acc: 0.9237, Val Loss: 0.6447, Val Acc: 0.7214
Epoch 8/10, Loss: 0.2371, Acc: 0.8900, Val Loss: 0.5951, Val Acc: 0.7512
Epoch 9/10, Loss: 0.1004, Acc: 0.9700, Val Loss: 0.7894, Val Acc: 0.7264
Epoch 10/10, Loss: 0.0726, Acc: 0.9838, Val Loss: 0.7100, Val Acc: 0.7214


# Question 3 <br>
What is the median of training accuracy for all the epochs for this model?<br>
<br>
0.05<br>
0.12<br>
0.40<br>
0.84<br>

In [30]:
median_train_acc = np.median(history['acc'])
print("Std training loss:", median_train_acc)


Std training loss: 0.8775


Question 4
What is the standard deviation of training loss for all the epochs for this model?

0.007
0.078
0.171
1.710

In [32]:
std_train_loss = np.std(history['loss'])
print("Median training acc:", std_train_loss)

Median training acc: 0.17994379536922864


# Data Augmentation
For the next two questions, we'll generate more data using data augmentations.<br>
<br>
Add the following augmentations to your training data generator:<br>
<br>
transforms.RandomRotation(50),<br>
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),<br>
transforms.RandomHorizontalFlip(),<br>

In [25]:
aug_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset_aug = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=aug_transforms)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)


# Question 5<br>
Let's train our model for 10 more epochs using the same code as previously.<br>

Note: make sure you don't re-create the model. we want to continue training the model we already started training.<br>
<br>
What is the mean of test loss for all the epochs for the model trained with augmentations?<br>

0.008<br>
0.08<br>
0.88<br>
8.88<br>

In [26]:
num_epochs_aug = 10
test_loss_list = []
test_acc_list = []

for epoch in range(num_epochs_aug):
    model.train()
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)

            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    mean_test_loss = val_running_loss / len(validation_dataset)
    mean_test_acc = correct_val / total_val

    test_loss_list.append(mean_test_loss)
    test_acc_list.append(mean_test_acc)

    print(f"[AUG] Epoch {epoch+1}/{num_epochs_aug} "
          f"Val Loss: {mean_test_loss:.4f}, Val Acc: {mean_test_acc:.4f}")


[AUG] Epoch 1/10 Val Loss: 0.4729, Val Acc: 0.7662
[AUG] Epoch 2/10 Val Loss: 0.5236, Val Acc: 0.7413
[AUG] Epoch 3/10 Val Loss: 0.4712, Val Acc: 0.7761
[AUG] Epoch 4/10 Val Loss: 0.4787, Val Acc: 0.8010
[AUG] Epoch 5/10 Val Loss: 0.5387, Val Acc: 0.7363
[AUG] Epoch 6/10 Val Loss: 0.4692, Val Acc: 0.7761
[AUG] Epoch 7/10 Val Loss: 0.4598, Val Acc: 0.7960
[AUG] Epoch 8/10 Val Loss: 0.5002, Val Acc: 0.7711
[AUG] Epoch 9/10 Val Loss: 0.4556, Val Acc: 0.8308
[AUG] Epoch 10/10 Val Loss: 0.5240, Val Acc: 0.7313


In [28]:
mean_test_loss_aug = np.mean(test_loss_list)
print("Mean test loss w/ aug:", mean_test_loss_aug)

Mean test loss w/ aug: 0.4893853720221946


# Question 6<br>
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?<br>
<br>
0.08<br>
0.28<br>
0.68<br>
0.98<br>

In [29]:
last5_test_acc = np.mean(test_acc_list[5:10])
print("Avg test acc last 5 epochs:", last5_test_acc)

Avg test acc last 5 epochs: 0.7810945273631841
